In [2]:
from typing import TypedDict, Annotated
from langgraph.graph import StateGraph, END
from langgraph.checkpoint.memory import MemorySaver
from langchain_ollama import ChatOllama
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
import operator

In [3]:
# Define the state - Simple version!
class ChatState(TypedDict):
    messages: list  # Just a regular list

In [4]:
# Initialize the LLM (using Ollama with llama3.2)
llm = ChatOllama(
    model="llama3.2",  # or "llama3.2:1b", "mistral", "phi3"
    temperature=0.7,
)

In [ ]:
def chatbot_node(state: ChatState) -> ChatState:
    """Main chatbot node that calls the LLM"""
    # Get all messages from state
    messages = state["messages"]
    
    # Call the LLM
    response = llm.invoke(messages)
    
    # Manually append the response to existing messages
    updated_messages = messages + [response]
    
    # Return the complete updated message list
    return {"messages": updated_messages}


def create_chatbot():
    """Build the chatbot graph"""
    # Create the graph
    workflow = StateGraph(ChatState)
    
    # Add the chatbot node
    workflow.add_node("chatbot", chatbot_node)
    
    # Set entry point
    workflow.set_entry_point("chatbot")
    
    # Add edge to END
    workflow.add_edge("chatbot", END)
    
    # Compile with memory to maintain conversation history
    memory = MemorySaver()
    app = workflow.compile(checkpointer=memory)
    
    return app

In [6]:
def chat_loop():
    """Interactive chat loop"""
    print("=" * 60)
    print("🤖 Simple Chatbot with LangGraph + Ollama")
    print("=" * 60)
    print("Type 'quit', 'exit', or 'bye' to end the conversation")
    print("=" * 60)
    print()
    
    # Create the chatbot
    app = create_chatbot()
    
    # Configuration for conversation thread
    config = {"configurable": {"thread_id": "user_session_1"}}
    
    # System message to set the chatbot's behavior
    conversation_history = [
        SystemMessage(content="You are a helpful, friendly AI assistant. Keep your responses concise and engaging.")
    ]
    
    while True:
        # Get user input
        user_input = input("You: ").strip()
        
        # Check for exit commands
        if user_input.lower() in ['quit', 'exit', 'bye', 'goodbye']:
            print("\n🤖 Chatbot: Goodbye! Have a great day! 👋")
            break
        
        # Skip empty inputs
        if not user_input:
            continue
        
        # Add user message to conversation history
        conversation_history.append(HumanMessage(content=user_input))
        
        # Create state with full conversation history
        current_state = {"messages": conversation_history}
        
        # Get chatbot response
        try:
            result = app.invoke(current_state, config)
            
            # Update conversation history with the result
            conversation_history = result["messages"]
            
            # Display the AI's response (last message)
            ai_message = conversation_history[-1]
            print(f"\n🤖 Chatbot: {ai_message.content}\n")
            
        except Exception as e:
            print(f"\n❌ Error: {e}")
            print("Make sure Ollama is running and the model is downloaded.")
            print("Run: ollama pull llama3.2")
            break

# Simple single question function
def ask_question(question: str, model: str = "llama3.2"):
    """Ask a single question without conversation history"""
    app = create_chatbot()
    config = {"configurable": {"thread_id": "single_qa"}}
    
    state = {
        "messages": [
            SystemMessage(content="You are a helpful AI assistant."),
            HumanMessage(content=question)
        ]
    }
    
    result = app.invoke(state, config)
    return result["messages"][-1].content


In [ ]:
chat_loop()